In [1]:
import pyspark
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

## Green

In [15]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [16]:
df_green.registerTempTable('green')

In [40]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    -- Revenue calculation 
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records

FROM green
WHERE lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 1, 2
ORDER BY 1,2    
""")

In [41]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 368.7800000000001|            18|
|2020-01-01 00:00:00|  16|              20.3|             1|
|2020-01-01 00:00:00|  17|58.300000000000004|             3|
|2020-01-01 00:00:00|  25|               7.2|             1|
|2020-01-01 00:00:00|  31|             23.34|             1|
|2020-01-01 00:00:00|  35|109.27000000000001|             2|
|2020-01-01 00:00:00|  36|            144.93|             8|
|2020-01-01 00:00:00|  37|160.89000000000001|             5|
|2020-01-01 00:00:00|  39|             66.22|             2|
|2020-01-01 00:00:00|  40|              13.8|             1|
|2020-01-01 00:00:00|  41|             50.45|             4|
|2020-01-01 00:00:00|  42| 94.16999999999999|            10|
|2020-01-01 00:00:00|  56|              8.84|             1|
|2020-01-01 00:00:00|  6

In [42]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

## Yellow

In [27]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [28]:
df_yellow.registerTempTable('yellow')

In [43]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    -- Revenue calculation 
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records

FROM yellow
WHERE tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 1, 2
ORDER BY 1,2    
""")

In [44]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

## Join Green + Yellow

In [49]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [50]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [51]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [52]:
df_join.show()

+-------------------+----+------------+--------------------+------------------+---------------------+
|               hour|zone|green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   4|        null|                null|            499.22|                   27|
|2020-01-01 00:00:00|  26|        null|                null|               3.8|                    1|
|2020-01-01 00:00:00|  56|        8.84|                   1|             73.67|                    1|
|2020-01-01 00:00:00|  61|       28.05|                   4|             18.76|                    2|
|2020-01-01 00:00:00|  65|        null|                null|            113.66|                    5|
|2020-01-01 00:00:00|  74|       83.68|                   7|            279.88|                   24|
|2020-01-01 00:00:00|  88|        null|                null|117.30000000000001|   

In [48]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')


In [53]:
df_join = spark.read.parquet('data/report/revenue/total')

In [54]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  22|              null|                null|               0.0|                    2|
|2020-01-01 00:00:00|  35|109.27000000000001|                   2|             15.75|                    1|
|2020-01-01 00:00:00|  40|              13.8|                   1|             25.86|                    3|
|2020-01-01 00:00:00|  43|              null|                null|387.55000000000007|                   19|
|2020-01-01 00:00:00|  70|              null|                null|               9.8|                    1|
|2020-01-01 00:00:00|  76|              null|                null|             75.81|                    2|
|2020-01-01 00:00:00|  77|  

In [55]:
df_zones = spark.read.parquet('zones')

In [56]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [59]:
df_result.drop('LocationId', 'zone').write.parquet('tmp/revenue-zones')